<a href="https://colab.research.google.com/github/MengOonLee/Deep_learning/blob/master/Diffusion/Sampling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab 1, Sampling

In [ ]:
# %%writefile diffusion_utilities.py
import numpy as np
import torch

class ResidualConvBlock(torch.nn.Module):


# Setting Things Up

In [ ]:
# hyperparameters
import torch

# diffusion hyperparameters
timesteps = 500
beta1 = 1e-4
beta2 = 0.02

# network hyperparameters
device = torch.device(device="cuda:0" if torch.cuda.is_available()
    else torch.device('cpu'))
n_feat = 64 # 64 hidden dimension feature
n_cfeat = 5 # context vector is of size 5
height = 16 # 16x16 image
save_dir= './weights/'

In [ ]:
import torch

# construct DDPM noise schedule
b_t = beta1 + (beta2 - beta1) * torch.linspace(
    start=0, end=1, steps=timesteps + 1, device=device)
a_t = 1 - b_t
ab_t = torch.cumsum(input=a_t.log(), dim=0).exp()
ab_t[0] = 1

In [ ]:
# construct model
import torch

class ContextUnet(torch.nn.Module):
    # cfeat - context features
    def __init__(self, in_channels, n_feat=256, n_cfeat=10, height=28):
        super(ContextUnet, self).__init__()

        # number of input channels, number of intermediate feature maps
        # and number of classes
        self.in_channels = in_channels
        self.n_feat = n_feat
        self.n_cfeat = n_cfeat
        # assume h == w. must be devisible by 4, so 28, 24, 20, 16...
        self.h = height

        # Initialize the initial convolutional layer
        self.init_conv = ResidualConvBlock(in_channels, n_feat, is_res=True)

        # Initialize the down-sampling path of the U-Net with two levels
        self.down1 = UnetDown(n_feat, n_feat)
        self.down2 = UnetDown(n_feat, 2 * n_feat)

nn_model = ContextUnet(in_channels=3, n_feat=n_feat, n_cfeat=n_cfeat,
    height=height).to(device)